# Version multidimensionnelle (Piterbarg 2 p.575)


On pose $$\chi(t) = - \frac{\mathrm{d}H(t)}{\mathrm{d}t} H(t)^{-1}$$ Et $H(t) = diag(h(t)) = diag\big( (h_1(t),...,h_d(t) )\big)$

Donc $$\chi(t) = \begin{pmatrix} -h_1'(t) & 0 \\ ... & ... \\ 0 .. & -h_d'(t)\end{pmatrix} \begin{pmatrix} h_1(t) & 0 \\ ... & ... \\ 0 .. & h_d(t)\end{pmatrix}^{-1} \\
\iff \chi(t) = \begin{pmatrix} - \frac{h_i'(t)}{h_i(t)} \mathbb{1}_{i=j} \end{pmatrix}$$

Donc $$\forall i=1,d, \quad h_i(t) = e^{-\chi_i t}$$

Avec cela, $$H(t) H^f(t)^{-1} = \big(H^f(t) H(t)^{-1} \big)^{-1} = \begin{bmatrix} e^{\chi_1 t}e^{-\chi_1 (t+\delta_1)} & ... & e^{\chi_d t} e^{-chi_d (t+\delta_1)} \\
 & ... & \\
e^{\chi_1 t}e^{-\chi_1 (t+\delta_d)} & ... & e^{\chi_d t}e^{-\chi_d (t+\delta_d)} \end{bmatrix}^{-1} \\
= \begin{bmatrix} e^{-\chi_1 \delta_1} & ... & e^{-\chi_d \delta_1} \\
 & ... & \\
e^{-\chi_1 \delta_d} & ... & e^{-\chi_d \delta_d} \end{bmatrix}^{-1}$$

Enfin on pose la volatilité locale: $$\sigma_r(t,x(t),y(t))^\top = H(t) H^f(t)^{-1} \begin{pmatrix} \lambda_1(a_1 + b_1 f_1(t)) & 0 & 0.. \\ 0 & \lambda_2(a_2 + b_2 f_2(t)) & 0 ..\\ 0 & .. & \lambda_d(a_d + b_d f_d(t))\end{pmatrix} \ D^f(t)^\top \\
= \tilde{H}(t) \ \sigma^f(t,f(t)) \ D^f(t)^\top$$

Où $D^f(t)^\top$ est telle que $$\Gamma = D^f(t)^\top D^f(t)$$ avec $\Gamma$ la matrice de corrélation des taux $f_i(t)$.

Enfin, $$\forall i =1,d, \quad f_i(t) = f(t,t + \delta_i) = f(0,t+\delta_i) + \mathbf{1}^\top H(t+ \delta_i) H(t)^{-1} \big(x(t) + y(t) \ G(t,t+\delta_i)\big)$$

$$G(t,T) = \int_{t}^{T} H(u) H(t)^{-1} \mathbf{1} \ \mathrm{d}u \\
= \begin{bmatrix} \frac{1 - e^{- \chi_1(T-t)}}{\chi_1} \\ ... \\ \frac{1 - e^{- \chi_d(T-t)}}{\chi_d} \end{bmatrix}$$

In [1]:
import numpy as np
import math
from time import time

def vecteur_uni(d):
    return np.transpose(np.ones(d))

def diag(v):
    n = len(v)
    mat = np.zeros((n,n))
    for i in range(n):
        mat[i][i] = v[i]
    return mat

In [2]:
#return cholesky decomposition D of the correlation matrix
def D(corr_mat):
    return np.linalg.cholesky(corr_mat)

def H(t,chi,d):
    v = [np.exp(-chi[i]*t) for i in range(d)]
    return diag(v)

def H_tilde(chi,delta,d):
    resu = np.zeros((d,d))
    for i in range(d):
        for j in range(d):
            resu[i][j] = np.exp(-chi[j] * delta[i])
    return np.linalg.inv(resu)

def G(t,T,chi,d):
    resu = np.zeros(d)
    for i in range(d):
        resu[i] = (1-np.exp(-chi[i]*(T-t)))/chi[i]
    return np.transpose(resu)
    
    
# Return the forward rates f1(t),f2(t)... fd(t) 
#init_forward : vector f(0,t+delta1),... f(0,t+deltad)

def f(t,chi,delta,d,init_forward,x,y):
    resu = np.zeros(d)
    Gt = np.array([G(t,t+delta[i],chi,d) for i in range(d)])
    for i in range(d):
        vect1 = np.array([np.exp(-chi[j]*delta[i]) for j in range(d)])
        vect2 = x + y.dot(Gt[i])
        resu[i] = init_forward[i] + vect1.dot(vect2)
    return resu

$$r(t) = f(0,t) + \mathbb{1}^\top x(t) \\ \implies e^{- \int_{0}^{T_0} r(s) \ \mathrm{d}s} = P(0,T_0) \ e^{- \sum_{i=1}^d \int_{0}^{T_0} x^{(i)}(s) \ \mathrm{d}s} = P(0,T_0) \ e^{\sum_{i=1}^d I_i(T_0)}$$

Simulation des trajectoires de $x,y,I$ en dimension $d$.
La fonction renvoit M triplets du type $$\begin{bmatrix} x_1(T_0) & ... & x_d(T_0) \end{bmatrix} ,  
\begin{bmatrix} y_{1,1}(T_0) & ... & y_{1,d}(T_0) \\
... \\
y_{d,1}(T_0) & ... & y_{d,d}(T_0) \end{bmatrix} , 
\begin{bmatrix} I_1(T_0) & ... & I_d(T_0)\end{bmatrix}$$

In [3]:
#Limite supérieure des diffusions de x_1,...,x_d : limiter artificiellement les elements à 1e10 pour éviter les explosions
def lim_sup_diff(d,x):
    for i in range(d):
        if(x[i]>1e10):
            x[i] = 1e10
    return x

#Simulation du schéma discret
def simul_multi(M,N,T0,d,chi,delta,init_forward,corr_mat,lmbda,a,b):
    dt = T0/N
    resu_x = np.zeros((M,d))
    resu_y = np.zeros((M,d,d))
    resu_I = np.zeros((M,d))
    v_uni = vecteur_uni(d)
    chi_mat = diag(chi)
    for m in range(M):
        I = np.zeros(d)
        x = np.zeros(d)
        y = np.zeros((d,d))
        Z = np.random.standard_normal(N*d)
        for i in range(N):
            t = i * dt
            #construction de sigma
            H_tild = H_tilde(chi,delta,d)
            ft = f(t,chi,delta,d,init_forward,x,y)
            diag_sigmaf = [lmbda[k] * (a[k] + b[k]*ft[k]) for k in range(d)]
            sigma_f = diag(diag_sigmaf)
            Df = D(corr_mat)
            sigma_i = H_tild.dot(sigma_f.dot(np.transpose(Df)))
            # incrément gaussien
            incr = math.sqrt(dt)*np.array([Z[i*d+j] for j in range(d)])

            I = I - x * dt
            
            x = x + (y.dot(v_uni) - chi_mat.dot(x)) * dt + sigma_i.dot(incr)
            x = lim_sup_diff(d,x) #On prend gare à l'explosion d'un terme
            
            y = y + (sigma_i.dot(np.transpose(sigma_i)) - chi_mat.dot(y) - y.dot(chi_mat)) * dt
        resu_I[m] = I
        resu_x[m] = x
        resu_y[m] = y
    return resu_I,resu_x,resu_y

In [4]:
def payoff_swaption_multi(d,maturities,bonds,x,y,chi,K):
    T0 = maturities[0]                             #maturities = [T0,...,TN]
    nb_maturities = len(maturities)
    A = 0
    #calculate the annuity A
    for n in range(nb_maturities-1):
        g = G(T0,maturities[n+1],chi,d)
        A += (maturities[n+1]-maturities[n])*bonds[n+1]*np.exp(-1*g.dot(x)-0.5*g.dot(y.dot(g)))
    g = G(T0,maturities[-1],chi,d)
    swap = bonds[0] - bonds[-1]*np.exp(-1*g.dot(x)-0.5*g.dot(y.dot(g))) - K*A
    if swap>0:
        return swap
    else:
        return 0

In [5]:
def swaption_MC_multi(d,M,sim,K,chi,bonds,maturities,exec_time=False,variance = False):
    t1 = time()
    Monte_Carlo = 0
    moment_2 = 0
    I,X,Y = sim
    for m in range(M):
        int_x = I[m]
        x = X[m]
        y = Y[m]
        s_m = np.exp(sum(int_x))*payoff_swaption_multi(d,maturities,bonds,x,y,chi,K)
        Monte_Carlo += s_m
        moment_2 += s_m**2
    t2 = time()
    price = Monte_Carlo/M
    if(exec_time):
        print("Execution time: ",t2-t1, "sec")
    if(variance):
        var = moment_2/M - price**2
        print("Variance:",var)
    return price

## Tests

In [12]:
d = 2
chi = [1,0.5]
delta = [1,2]

print("H_tilde=",H_tilde(chi,delta,d))

print(G(0,1,chi,2))

H_tilde= [[  6.90849718 -11.39018625]
 [ -2.54149408   6.90849718]]
[0.63212056 0.78693868]


In [13]:
M = 1000
N = 250
T0 = 1
init_forward = [0.05,0.05,0.05]
corr_mat = np.eye(d)
a = [0,0,0]
b = [0.5,0.5,0.5]
lmbda = [1,1,1]
I,X,Y = simul_multi(M,N,T0,d,chi,delta,init_forward,corr_mat,lmbda,a,b)

In [8]:
print(X,Y)

[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 ...
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]] [[[nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 ...

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]]]


In [9]:
y = np.zeros((3,3))
x = np.array([1,1,1])
K = 0.05
maturities = [1,2,3]
bonds = [np.exp(-0.05*m) for m in maturities]
payoff_swaption_multi(d,maturities,bonds,x,y,chi,K)

0.9001271274175793

In [10]:
#Temps d'execution
for m in [1000,10000,100000]:
    t_start = time()
    sim = simul_multi(m,N,T0,d,chi,delta,init_forward,corr_mat,lmbda,a,b)
    prix = swaption_MC_multi(d,m,sim,K,chi,bonds,maturities,True,True)
    t_finish = time()
    print("Temps pour M= " +str(m) + ":",t_finish-t_start, "Prix = ",prix)
    print()

Execution time:  0.03590130805969238 sec
Variance: nan
Temps pour M= 1000: 31.325398921966553 Prix =  nan

Execution time:  0.39594197273254395 sec
Variance: nan
Temps pour M= 10000: 306.3957200050354 Prix =  nan

Execution time:  3.9873998165130615 sec
Variance: nan
Temps pour M= 100000: 3158.533299446106 Prix =  nan



| M | Temps (sec) | Variance |
|--------------|-----------|-----------|
| 1000         | 24.2     |    |
| 10 000       |  262.7    |     |
| 100 000      | 2454.7    |   |